# Tutorial 4 - Working with PAGA and velocyto

## Load required packages

In [ ]:
import warnings
warnings.simplefilter(action = "ignore", category = FutureWarning)
import os
import sys
import re
import copy
import loompy
import glob
import igraph
import sklearn as sk
import scipy as sp
import velocyto as vcy
import scanpy as sc
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from igraph import *
from numpy_groupies import aggregate, aggregate_np
from matplotlib import colors
sc.logging.print_header()
sc.settings.n_jobs = 4

In [ ]:
pwd = os.getcwd()

In [ ]:
pwd

## PAGA

By quantifying the connectivity of partitions (groups, clusters) of the single-cell graph, partition-based graph abstraction (PAGA) generates a much simpler abstracted graph (PAGA graph) of partitions, in which edge weights represent confidence in the presence of connections.

See https://scanpy.readthedocs.io/en/stable/api/scanpy.tl.paga.html for more usage information

### Data ingest

In [ ]:
# Read in the loom file with scanpy as we did the h5ad file from last week
adata = sc.read_loom(os.path.join(pwd, "MACA_bonemarrow_10x.loom"))
adata.var_names_make_unique()

# The loom file came from velocyto's preprocessing tool, which splits the UMIs
# into three groups: those from spliced transcripts, unspliced transcripts
# those that are ambiguous. By default .X is set to the spliced UMIs,
# we reset that here to include all UMIs for PAGA
adata.X = adata.layers["matrix"].copy()

# Create a gene metadata column called 'Gene' that stores the gene symbols
# This is for velocyto later
adata.var['Gene'] = adata.var.index

adata

In [ ]:
# Read in the cellular metadata
adata.obs = pd.read_csv(os.path.join(pwd, "MACA_bonemarrow_10x_metadata.txt"),
                        index_col=3,
                        delimiter="\t")

# Remove an unused index column
adata.obs.drop('phenotype', axis=1, inplace=True)

# Rename the columns "order" and "loomID" metadata columns to more informative names
adata.obs.rename({'order': 'free_annotation', 'loomID': 'cluster'}, axis=1, inplace=True)

# Create a cell metadata column called 'CellID' that stores the cell names
# This is for velocyto later
adata.obs['CellID'] = adata.obs.index

# Creates a cell metadata column called "SampleID" that stores the 10x channel
# each cell came from. We. use the sub() function from the re library to obtain
# the channel information with a list comprehension loop
adata.obs['SampleID'] = [re.sub('^(10X_P[0-9]{1,2}_[0-9]{1,2})_possorted_genome_.*$', '\\1', i) for i in adata.obs.index]
adata.obs

In [ ]:
# Save a copy of adata for later subsetting
adata_raw = copy.deepcopy(adata)

In [ ]:
# Keep only monocytes and granulocytes and their progenitors
# This simplifies the trajectory analysis so we can more easily
# apply all four tools across the same dataset
adata = adata[[i in ['Monocytes', 'Monocyte_progenitors', 'Stem_Progenitors', 'Granulocyte_progenitors', 'Granulocytes'] for i in list(adata.obs.free_annotation)]]
adata_raw = adata_raw[[i in ['Monocytes', 'Monocyte_progenitors', 'Stem_Progenitors', 'Granulocyte_progenitors', 'Granulocytes'] for i in list(adata_raw.obs.free_annotation)]]
adata

### scanpy clustering

In [ ]:
plt.rcParams['figure.figsize'] = [12, 6]

# Run through the boilerplate scanpy procedure from normalization
# to feature selection to principle component analysis from last week
sc.pp.normalize_total(adata, target_sum=1e4)
sc.pp.log1p(adata)
adata.raw = adata
sc.pp.highly_variable_genes(adata, min_mean=0.0125, max_mean=6, min_disp=0.5)
sc.pl.highly_variable_genes(adata)
sc.pp.scale(adata, max_value=10)
sc.tl.pca(adata, svd_solver='arpack', n_comps=30)
sc.pl.pca_variance_ratio(adata, n_pcs=30)

In [ ]:
plt.rcParams['figure.figsize'] = [10, 10]

# Cluster the data using the same approach as last week
sc.pp.neighbors(adata, n_neighbors=50, n_pcs=8)
sc.tl.leiden(adata, resolution=0.25)

# Reduce the principle compnents used in clustering down to 2 dimensions with the
# ForceAtlas2 algortithm. This step is conceptually similar to UMAP, but is 
# optimized for trajectories and dynamic processes. See Jacomy et al (2014) PLoS One
# For more information about the algorithm
sc.tl.draw_graph(adata)

# Plot cells based on their coordinates from ForceAtlas2, paint on the
# free_annotation's from MACA and the leiden clustering
sc.pl.draw_graph(adata, color=["free_annotation", "leiden"], legend_loc="on data", legend_fontsize=14)

**Question:** Do the Leiden clusters align with the cellular annotations?

**Answer:**

In [ ]:
# Calculate genes enriched in each cellular annotation
sc.tl.rank_genes_groups(adata, groupby="free_annotation", method="wilcoxon")

In [ ]:
plt.rcParams['figure.figsize'] = [12, 12]
sc.set_figure_params(fontsize=12)

# Plot expression of 4 markers per annotation
sc.pl.draw_graph(adata, color=np.ravel(np.array(pd.DataFrame(adata.uns['rank_genes_groups']['names']).head(4).T)), size=50)

**Question:** Do any of the groups of genes identified seem to represent technical axes of variation? If so, which one and what is the artifact?

**Answer:**

In [ ]:
# Calculate genes enriched in each leiden cluster
sc.tl.rank_genes_groups(adata, groupby="leiden", method="wilcoxon")

In [ ]:
plt.rcParams['figure.figsize'] = [12, 12]
sc.set_figure_params(fontsize=12)

# Plot expression of 4 markers per cluster
sc.pl.draw_graph(adata, color=np.ravel(np.array(pd.DataFrame(adata.uns['rank_genes_groups']['names']).head(4).T)), size=50)

**Question:** Do any of the annotations seem to represent more than one cell type? If so, which one? 

**Answer:**

In [ ]:
# Python dictionaries allow us to store information in keys, for exmaple:
# my_dict = {'key': value, 'science': 'is', 'really': 'fun'}
# my_dict['science'] will return "is" and my_dict['really'] will return "fun"
# In this case we'll use a dict where each "key" is a cell type and the information stored
# (the "value") is a data frame of differentially expressed genes for that cell type
marker_genes = {}

    
# Python has something called comprehension that allows for 1-liner loops
# In the simplest case, something like [print(x) for x in [1, 2, 3]] would print 1, 2 and 3 on separate lines.
# It is equivalent to
#
# for x in [1, 2, 3]:
#    print(x)
#
# In this case, we use two nested loops
# The inner loop, for key in ['names', 'pvals_adj', 'logfoldchanges'], pulls the gene symbols, adjusted p-values,
# and logfoldchanges from the differential gene expression results in the temporary adata object for each
# cell type, which is set by the outer loop, for group in groups. The outer loop then stores the results of the inner
# loop to a temporary dictionary with the cell types name as the key.
result = adata.uns['rank_genes_groups']
groups = result['names'].dtype.names
marker_genes = {group: pd.DataFrame({key: result[key][group] for key in ['names', 'pvals_adj', 'logfoldchanges']}) for group in groups}

In [ ]:
# Plot the top 25 genes for any cluster, right now cluster 5 is shown
marker_genes['6'].iloc[:50,:]

**Question:** Are there clusters that appear off of the main trajectory from HSCs to either granulocytes or monocytes? Hint: Look at the first code block below.

**Answer**: 

### PAGA and pseudotime calculation

In [ ]:
# If there are off-trajectory cells, remove them. If you want to remove clusters
# 4 and 7 change ['clusters', 'to', 'remove'] to ['4', '7']
adata = adata_raw[[i not in ['4', '7'] for i in list(adata.obs.leiden)]]

In [ ]:
plt.rcParams['figure.figsize'] = [12, 6]

# Run through the boilerplate scanpy procedure from normalization
# to feature selection to principle component analysis from last week
# without the off-trajectory clusters
sc.pp.normalize_total(adata, target_sum=1e4)
sc.pp.log1p(adata)
adata.raw = adata
sc.pp.highly_variable_genes(adata, min_mean=0.0125, max_mean=6, min_disp=0.5)
sc.pl.highly_variable_genes(adata)
sc.pp.scale(adata, max_value=10)
sc.tl.pca(adata, svd_solver='arpack', n_comps=30)
sc.pl.pca_variance_ratio(adata, n_pcs=30)

In [ ]:
plt.rcParams['figure.figsize'] = [10, 10]

# Re-run neighbors, leiden, and ForceAtlas2 without the off-trajectory cells
sc.pp.neighbors(adata, n_neighbors=50, n_pcs=7)
sc.tl.leiden(adata, resolution=0.25)
sc.tl.draw_graph(adata)

# Plot the cells with the new ForceAtlas2 coordinates
sc.pl.draw_graph(adata, color=["free_annotation", "leiden"], legend_loc="on data", legend_fontsize=14)

In [ ]:
plt.rcParams['figure.figsize'] = [10, 10]

# Run PAGA, with it aware of the leiden clustering
# See. 
sc.tl.paga(adata, groups="leiden")

# Plot the relationships and their strengths between each cluster
sc.pl.paga(adata)

In [ ]:
plt.rcParams['figure.figsize'] = [10, 10]

# Re-run and re-plot ForceAtlas2 with it aware of the PAGA relationships
sc.tl.draw_graph(adata, init_pos='paga')
sc.pl.draw_graph(adata, color=['free_annotation', 'leiden'], legend_loc="on data", legend_fontsize=14)

In [ ]:
plt.rcParams['figure.figsize'] = [10, 10]

# Set the starting cell cluster to 5 (which corresponds with HSCs)
# This is required so the pseudotime algortihm below
# See https://scanpy.readthedocs.io/en/stable/api/scanpy.tl.dpt.html#scanpy.tl.dpt
# for more usage information on sc.tl.dpt
adata.uns['iroot'] = np.flatnonzero(adata.obs.leiden  == '5')[0]
sc.tl.dpt(adata, n_dcs=7)

# Plot the pseudotime on revised ForceAtlas2 coordinates
sc.pl.draw_graph(adata, color=['dpt_pseudotime'], legend_loc="on data", legend_fontsize=14)

In [ ]:
plt.rcParams['figure.figsize'] = [6, 6]
# Plot some differentially expressed genes from above that were along the trajectory
sc.pl.draw_graph(adata, color=["Ctss", "Lgals1", "Ctsg", "Hmgn2", "Mmp8"], legend_loc="on data", legend_fontsize=14, ncols=2)

**Question:** Conceptually, how could you use pesudotime and the PAGA trajectories to to refine your differential gene expression analysis?

**Answer:**

In [ ]:
# Write the PAGA loom file to disk for use with velocity
adata.write_loom(os.path.join(pwd, "MACA_bonemarrow_10x_PAGA.loom"))

In [ ]:
# Write out UMI table and cellular and gene metadata for monocle3 later
# Opening a loom file in R, while possible, has gotten more difficult recently
# as their use has waned and support diminshed by major packages
UMIs = pd.DataFrame(data=adata.layers["matrix"].T.todense(),
                    columns=adata.obs_names, 
                    index=adata.var_names)
UMIs['Genes'] = UMIs.index
UMIs = UMIs.groupby('Genes').sum()
UMIs.to_csv(os.path.join(pwd, 'MACA_bonemarrow_10x.csv'))

adata.obs.to_csv(os.path.join(pwd, 'MACA_bonemarrow_10x_obs.csv'))

adata.var[~adata.var.Gene.duplicated()].to_csv(os.path.join(pwd, 'MACA_bonemarrow_10x_var.csv'))

## Velocyto

RNA velocity is a high-dimensional vector that predicts the future state of individual cells on a timescale of hours. RNA velocity is the time derivative of the gene expression state and can be directly estimated by distinguishing between unspliced and spliced mRNAs in common single-cell RNA sequencing protocols. See Manno et al (2018) _Nature_ for more details

And http://velocyto.org/velocyto.py/index.html for usage information. We highly recommend you read their supplementary note on how the algorithm works. While technical, it lays out their model _very_ clearly. You can find it at https://static-content.springer.com/esm/art%3A10.1038%2Fs41586-018-0414-6/MediaObjects/41586_2018_414_MOESM1_ESM.pdf

### Data ingest

In [ ]:
# Load the loom file in as a velocyto object
vlm = vcy.VelocytoLoom(os.path.join(pwd, "MACA_bonemarrow_10x_PAGA.loom"))

### Explore the velocyto object

In [ ]:
# Print the cellular metadata
vlm.ca

In [ ]:
# Show available metadata columns
vlm.ca.keys()

In [ ]:
# Access a the free_annotation's for the first 5 cells
vlm.ca['free_annotation'][:5]

In [ ]:
# Access the gene metadata
vlm.ra

In [ ]:
# Show available gene metadata
vlm.ra.keys()

In [ ]:
# Access the mean expression of the first 5 genes
vlm.ra['mean'][:5]

In [ ]:
# Access the UMIs array from spliced
vlm.S

In [ ]:
# Access the UMIs array from unspliced
vlm.U

In [ ]:
# Access specific genes from spliced UMIs array from the first 5 cells
vlm.S[[i in ['Mpo', 'Ctsg'] for i in list(vlm.ra['Gene'])],:5]

In [ ]:
# Access specific genes from unspliced UMIs array from the first 5 cells
vlm.U[[i in ['Mpo', 'Ctsg'] for i in list(vlm.ra['Gene'])],:5]

### Normalize and filter spliced and unspliced gene expression

**Note:** The functions may look different from scanpy or Seurat, but this section is conceptually similar to how they both function

In [ ]:
# Tell velocyto what each cell's type is
vlm.set_clusters(vlm.ca['free_annotation'])

In [ ]:
# Plot the fraction of spliced, ambiguous, and unspliced UMIs in each 10x run
# Reads from unspliced pre-mRNA comes from ectopic poly-T priming
vlm.plot_fractions()

In [ ]:
# Normalize spliced and unspliced UMIs, the method is slightly different from scanpy
# and Seurat. Each spliced or unspliced UMI is multipled by a normalization factor, 
# which is the mean number of spliced or unspliced UMIs across all cells divided by 
# each individual cells total spliced or unspliced UMIs. Normalized UMIs are set to
# vlm.S_sz and vlm.U_sz
#
vlm.normalize("S", size=True, log=False)
vlm.normalize("U", size=True, log=False)

In [ ]:
# Filter genes out that have fewer than 30 spliced UMIs across all cells
# or that haved spliced expression in fewer than 20 cells
vlm.score_detection_levels(min_expr_counts=30, min_cells_express=20,
                           min_expr_counts_U=0, min_cells_express_U=0)
vlm.filter_genes(by_detection_levels=True)

In [ ]:
plt.rcParams['figure.figsize'] = [10, 10]


# Select the top 2000 genes based on their mean expression and variance
# Winsorizing is a statical transformation that limits extreme values
vlm.score_cv_vs_mean(2000, plot=True,
                     winsorize=True,
                     winsor_perc=(1,99.8),
                     svr_gamma=0.01)
vlm.filter_genes(by_cv_vs_mean=True)

In [ ]:
plt.rcParams['figure.figsize'] = [12, 6]

# Filter genes out that have fewer than 25 unspliced UMIs across all cells
# or that haved unspliced expression in fewer than 20 cells
vlm.score_detection_levels(min_expr_counts=0, min_cells_express=0,
                           min_expr_counts_U=25, min_cells_express_U=20)

vlm.filter_genes(by_detection_levels=True)

In [ ]:
# Normalize by total cell counts and log-transform the data
# The log-transformed values are stored in vlm.S_norm and vlm.U_norm
vlm.normalize_by_total(min_perc_U=0.5, plot=True)

# Adjust the splice count based on the relationship between total spliced and unspliced UMIs
vlm.adjust_totS_totU(normalize_total=True, fit_with_low_U=False, svr_C=1, svr_gamma=1e-04)

**Question:** How does the number of spliced UMIs compare with unspliced UMIs?

**Answer:**

In [ ]:
plt.rcParams['figure.figsize'] = [10, 10]

# Run principle component analysis and plot each components ratio of total variance
vlm.perform_PCA()
plt.plot(np.cumsum(vlm.pca.explained_variance_ratio_)[:100])
n_comps = np.where(np.diff(np.diff(np.cumsum(vlm.pca.explained_variance_ratio_))>0.0055))[0][0]
vlm.pcs[:,1] *= -1 # flip for consistency with previous version

In [ ]:
# For the preparation of the gamma fit we smooth the data using a kNN neighbors
# pooling approach. kNN neighbors can be calculated directly in gene expression
# space or reduced PCA space
#
# Creates smoothed splice and unspliced expression matrices on vlm.Sx and vlm.Ux
k=500
vlm.knn_imputation(n_pca_dims=n_comps,k=k, balanced=True,
                   b_sight=np.minimum(k*8, vlm.S.shape[1]-1),
                   b_maxl=np.minimum(k*4, vlm.S.shape[1]-1),
                   n_jobs=4)

In [ ]:
# Normalize to median cell sizes and then fit gamma to genes that
# survived the filtering step run. Gamma represents the rate of degredation
vlm.normalize_median()
vlm.fit_gammas(maxmin_perc=[2,95], limit_gamma=True)

### Model RNA velocity with first order rate kinetics

In [ ]:
# Predict U given the fitted gammas
vlm.predict_U()

# Calculate RNA velocity
vlm.calculate_velocity()

# Calculate the change in gene expression (dS/dt) assuming constant velocity
vlm.calculate_shift(assumption="constant_velocity")

# Extrapolate gene expression based on velocity
vlm.extrapolate_cell_at_t(delta_t=1)

# Set the display embedding as the first 2 principle components
vlm.Pcs = np.array(vlm.pcs[:,:2], order="C")

### Calculate new position of cells in chosen embedding

**Note:** In this case, we are going to display the cells on their first 2 principle components

In [ ]:
# Estimate the probability a cell represents a differentiation starting point
vlm.estimate_transition_prob(hidim="Sx_sz", embed="Pcs", transform="log", psc=1, n_neighbors=150, knn_random=True, sampled_fraction=1)

In [ ]:
# Use the transition probability to project the velocity direction on the embedding
vlm.calculate_embedding_shift(sigma_corr = 0.05, expression_scaling=True)

In [ ]:
# Calculate the vetor field for how cell expression changes across the embedding
vlm.calculate_grid_arrows(smooth=0.9, steps=(25, 25), n_neighbors=100)

In [ ]:
# Plot the cells in the first two principle components with the vector field of arrows overlaid
plt.figure(None,(10,10))
vlm.plot_grid_arrows(scatter_kwargs_dict={"alpha":0.7, "lw":0.7, "edgecolor":"0.4", "s":70, "rasterized":True},
                     min_mass=2.9, angles='xy', scale_units='xy',
                     headaxislength=2.75, headlength=5, headwidth=4.8, quiver_scale=0.35, scale_type="absolute")

# This inverts PC1, purely for stylistic reasons
plt.gca().invert_xaxis()

# This loops through the free annotations and plots a label at each groups median position
for i in vlm.ca["free_annotation"]:
    Pcs_m = np.median(vlm.Pcs[vlm.ca["free_annotation"] == i, :], 0)
    plt.text(Pcs_m[0], Pcs_m[1], str(vlm.ca["free_annotation"][vlm.ca["free_annotation"] == i][0]),
             fontsize=13, bbox={"facecolor":"w", "alpha":0.6})
    
# Removes axes
plt.axis("off")
plt.axis("equal");

In [ ]:
plt.rcParams['figure.figsize'] = [10, 10]

# Plot the cells that are upregulating (above the line) or downregulating
# each of the listed genes (from earlier differential gene expression analysis)
# The cells are colored by their annotation identity (as above).
vlm.plot_phase_portraits(["Ctss", "Lgals1", "Ctsg", "Hmgn2", "Mmp8"])

**Question:** Do the up- and down-regulation scores in the phase plots above make sense? Hint: It may be useful to compare with leiden cluster expression from earlier.

**Answer:**

### Find genes enriched as a function of pseudotime

In [ ]:
# Set the total number of cells
n_cells = vlm.cluster_ix.shape[0]

# Number of cells per cluster
sizes = np.bincount(vlm.cluster_ix)

# Number of nonzero values per cluster
nnz = aggregate(vlm.cluster_ix, (vlm.Sx_sz>0).astype(float), func="sum", axis=1)

# Mean value per cluster
means = aggregate(vlm.cluster_ix, vlm.Sx_sz, func="mean", axis=1)

# Non-zeros and means over all cells
nnz_overall, means_overall = (vlm.Sx_sz>0).sum(1), vlm.Sx_sz.mean(1)

# Scale by number of cells
f_nnz = nnz / sizes
f_nnz_overall = nnz_overall / n_cells

# Means and fraction non-zero values in other clusters (per cluster)
means_other = ((means_overall * n_cells)[None].T - (means * sizes)) / (n_cells - sizes)
f_nnz_other = ((f_nnz_overall * n_cells)[None].T - (f_nnz * sizes)) / (n_cells - sizes)

enrichment = (f_nnz + 0.1) / (f_nnz_overall[None].T + 0.1) * (means + 0.01) / (means_overall[None].T + 0.01)

In [ ]:
# Create dictionary of enriched genes
enriched = {}
for i in range(enrichment.shape[1]):
    tmp = pd.DataFrame([enrichment[:,i][np.argsort(enrichment[:,i])[::-1]][:350],vlm.ra["Gene"][np.argsort(enrichment[:,i])[::-1]][:350]]).T
    tmp.rename({0: 'score', 1: 'gene'}, inplace=True, axis=1)
    tmp.index = tmp.gene
    enriched[i] = tmp

In [ ]:
# Print table of top 25 enriched genes per identity
# 0 = Granulocyte progenitors, 1 = Granulocytes, 2 = Monocyte progenitors
# 3 = Monocytes, 4 = HSCs
enriched[4][:25]

In [ ]:
plt.figure(None,(10,10))
vlm.plot_expression_as_color('Ak4', imputed=True, which_tsne="Pcs")
plt.gca().invert_xaxis()

plt.figure(None,(10,10))
vlm.plot_expression_as_color('Chit1', imputed=True, which_tsne="Pcs")
plt.gca().invert_xaxis()

plt.figure(None,(10,10))
vlm.plot_expression_as_color('Tmem45a2', imputed=True, which_tsne="Pcs")
plt.gca().invert_xaxis()

plt.figure(None,(10,10))
vlm.plot_expression_as_color('Gria3', imputed=True, which_tsne="Pcs")
plt.gca().invert_xaxis()

plt.figure(None,(10,10))
vlm.plot_expression_as_color('Mx1', imputed=True, which_tsne="Pcs")
plt.gca().invert_xaxis()

### Further exploration of the velocyto object

In [ ]:
# Log-transformed, normalized spliced UMIs
vlm.S_norm

In [ ]:
# Normalized spliced UMIs
vlm.S_sz 

In [ ]:
# Smoothed spliced UMIs
vlm.Sx

In [ ]:
# Smoothed, normalized spliced UMIs
vlm.Sx_sz

In [ ]:
# Smoothed, normalized spliced UMIs adjusted based on their RNA velocity
vlm.Sx_sz_t

In [ ]:
# Fit gammas for each gene
vlm.gammas

In [ ]:
# Predicted unspliced UMIs based on RNA velocities
vlm.Upred

In [ ]:
# Coordinates for the visualization (first 2 principle components)
vlm.embedding

In [ ]:
# Shift for each cell based on predicted future transcriptome
vlm.delta_embedding

In [ ]:
# Per cell per gene RNA velocities as dS/dt
vlm.delta_S

**Question:** Conceptually, how could you use the embedding shifts or RNA velocities to refine your differntial gene expression analysis?

**Answer:**